In [233]:
#Se importan los paquetes necesarios para los cálculos.
import pandas as pd #manejo de datos
import numpy as np #parquete de matemática
import scipy.stats as sp #paquete de estadísticas
from sqlalchemy import create_engine #consultas a sql
import sqlalchemy #manejo de sql
import sys #manejo del sistema para recibir argumentos
import warnings #
from sqlalchemy import text
warnings.filterwarnings("ignore")
import pyodbc

import funciones

In [234]:
#Conexión a la base de datos
server = '10.0.0.153'
user = "testipc2"
pas = "Abc$2020"
#user = input('Ingrese el usuario: ')
#pas = input('Ingrese la contraseña: ')
try:
    #driver = 'SQL Server Native Client 11.0'
    driver = 'ODBC Driver 17 for SQL Server'
    database_connection = f'mssql://{user}:{pas}@{server}/master?driver={driver}'
    engine = create_engine(database_connection)
    connection = engine.connect()
    print('Conexión exitosa')
except:
    print('Fallo en la conexión')

Conexión exitosa


In [235]:
#aqui se ingresa el año y mes del indice que se va a calcular

anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))

#anio = 2024
#mes = 1
dec = 0

In [236]:
#aca se generan losnombres de los archivos que tienen el indice anterior y el indice actual
excel_indantmes = funciones.excel_mesant(mes)
excel_indantanio = funciones.excel_anioant(anio, mes)
excel_indact01 = 'indprod'
excel_indact02 = '.xlsx'

excel_indant = f"{excel_indact01}_{excel_indantmes}_{excel_indantanio}{excel_indact02}"
excel_indactual = f"{excel_indact01}_{mes}_{anio}{excel_indact02}"

#excel_indactual
excel_indant

'indprod_1_2024.xlsx'

In [237]:
#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Codigos(connect):
    querycod = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    
    return pd.read_sql(querycod, connect, index_col = 'ArtCod')

codigos = Codigos(connection)
codigos

""
ArtCod
011110101
011110102
011110103
011120101
011120102
...
125110101
125110201
125110301


In [238]:
#Se define una función para hallar cualquier conjunto de códigos.
#le asocia a la categoría su número en la base de datos
di = {'Art':6,'Gba':5,'Sub':4,'Gru':3,'Agr':2,'Div':1}
#le asocia a cada categoría la longitud de su código
leng = {'Div' : 2, 'Agr' : 3, 'Gru' : 4, 'Sub' : 5, 'Gba' : 7, 'Art': 9}

#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Nombres(datos, cod : str, connect = connection, dict = di):
    ''' 
    Esta función retorna un dataframe con los nombres y códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    if cod == 'Art':
        querycod = text(f"SELECT DISTINCT ArtCod, ArtNom FROM IPC2010_RN.dbo.IPC007")
    else:
        querycod = text(f'SELECT DISTINCT {cod}Cod, {cod}Nom FROM IPC2010_RN.dbo.IPCM0{dict[cod]}')

    nombres = pd.read_sql(querycod, connect, index_col = cod+'Cod')
    return pd.merge(nombres.reset_index(), datos.reset_index(), on = cod+'Cod',  how = 'inner').set_index(['RegCod',cod+'Cod'])

def CodigosCat(ind :str, cod, len = leng):
    ''' 
    Esta función retorna un dataframe con los códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    return pd.DataFrame([x[:len[ind]] for x in cod.index],columns=[ind+'Cod']).drop_duplicates().set_index(ind+'Cod')

In [239]:
#aqui se empieza la parte de la conexion con el otro server
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [240]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [241]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas01 = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")

Connection successful


In [242]:
boletas02 = boletas01
boletas02['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas02['decada'] = boletas02['decada'].replace(funciones.decadas)
boletas02['nt_tipo'] = boletas02['nt_tipo'].replace({351761:'S', 351762:'S'})
boletas02['nt_tipo'] = boletas02['nt_tipo'].fillna('N')
boletas02['cantidad_anterior'] = pd.to_numeric(boletas02['cantidad_anterior'], errors='coerce')
boletas02['cantidad_actual'] = pd.to_numeric(boletas02['cantidad_actual'], errors='coerce')
boletas02['precio_anterior'] = pd.to_numeric(boletas02['precio_anterior'], errors='coerce')
boletas02['precio_actual'] = pd.to_numeric(boletas02['precio_actual'], errors='coerce')
boletas02['decada'] = pd.to_numeric(boletas02['decada'], errors='coerce')
boletas02['region'] = boletas02['region'].astype(str)
boletas02['codigo_articulo'] = boletas02['codigo_articulo'].astype(str)
boletas02 = boletas02[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]
boletas02.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)
boletas02['RegCod'] = boletas02['RegCod'].replace({'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})

boletas02

,RegCod,ArtCod,BolNum,UraChi,UreCan,ArtPhi,ArtPac,ArtCR,PerSem
0,01,053110202,34-2-2024,1.0,1.0,145.0,145.0,N,1
1,01,031220302,36-2-2024,1.0,1.0,250.0,250.0,N,1
2,01,072110201,42-2-2024,1.0,1.0,400.0,400.0,N,1
3,01,041110103,43-2-2024,1.0,1.0,1700.0,1700.0,N,1
4,01,072110302,54-2-2024,946.0,946.0,70.0,70.0,N,1
...,...,...,...,...,...,...,...,...,...
55083,08,011711401,21871-2-2024,0.0,661.0,0.0,7.0,N,3
55084,08,011711101,21871-2-2024,0.0,454.0,0.0,10.0,N,3
55085,08,011711001,21871-2-2024,0.0,844.0,0.0,7.0,N,3
55086,08,1174051,21871-2-2024,0.0,1.0,0.0,1.0,N,3


In [243]:
boletas = boletas02[boletas02['ArtCod'].str.len() == 9]
boletas.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)
boletas.sort_index(inplace=True)
boletas

UraChi  UreCan   ArtPhi   ArtPac ArtCR  PerSem
RegCod ArtCod    BolNum                                                      
01     011110101 1035-2-2024    400.0   400.0     7.10     7.10     N       1
                 1104-2-2024    400.0   400.0     7.50     7.50     N       2
                 113-2-2024     400.0   400.0     7.15     7.15     N       1
                 1175-2-2024    400.0   400.0     7.10     7.10     N       2
                 1220-2-2024    400.0   400.0     7.10     7.10     N       1
...                               ...     ...      ...      ...   ...     ...
08     125110401 16025-2-2024     1.0     1.0  2500.00  2500.00     N       1
                 16027-2-2024     1.0     1.0  2000.00  2000.00     N       1
                 16052-2-2024     1.0     1.0  3800.00  3800.00     N       3
                 16435-2-2024     1.0     1.0  2800.00  2800.00     N       1
                 16444-2-2024     1.0     1.0  4000.00  4000.00     N       1

[51961 rows x 6 columns]

In [244]:
boletas03 = boletas01
boletas03['codigo_articulo'] = boletas01['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas03 = boletas03[['codigo_articulo', 'articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]

artimput = boletas03.rename(columns={'codigo_articulo':'ArtCod'})
artimput = artimput[artimput['ArtCod'].str.len() == 9]
#.value_counts()
artimput

,ArtCod,articulo,ine_poll_id,cantidad_anterior,cantidad_actual,precio_anterior,precio_actual,nt_tipo,decada
0,053110202,Olla y cacerola de 1 a 2lt,34-2-2024,1.0,1.0,145.0,145.0,N,1
1,031220302,Vestidos formales con mangas de largo hasta la...,36-2-2024,1.0,1.0,250.0,250.0,N,1
2,072110201,Llantas para automóvil 185/65 R14,42-2-2024,1.0,1.0,400.0,400.0,N,1
3,041110103,Alquiler de apartamento (con o sin servicios),43-2-2024,1.0,1.0,1700.0,1700.0,N,1
4,072110302,Aceite de motor sintético,54-2-2024,946.0,946.0,70.0,70.0,N,1
...,...,...,...,...,...,...,...,...,...
55082,011711301,Zanahorias frescos,21871-2-2024,0.0,1109.0,0.0,12.0,N,3
55083,011711401,Rábanos frescos,21871-2-2024,0.0,661.0,0.0,7.0,N,3
55084,011711101,Cebollas blancas,21871-2-2024,0.0,454.0,0.0,10.0,N,3
55085,011711001,Maíz verde fresco,21871-2-2024,0.0,844.0,0.0,7.0,N,3


In [245]:
codigos01 = codigos 
codigos01 = codigos01.reset_index()
#artimput = pd.DataFrame(artimput['ArtCod'].unique(), columns=['ArtCod'])
#artimput['ArtCod'] = artimput['ArtCod'].astype(str)
artimput = codigos01[~codigos01['ArtCod'].isin(artimput['ArtCod'])]
artimput

,ArtCod
1,011110102
15,011130104
17,011130301
37,011230102
38,011230103
...,...
428,122210101
429,122210201
430,122210301
433,123110103


In [246]:
boletas04 = boletas01[['codigo_articulo', 'articulo']].rename(columns={'codigo_articulo':'ArtCod'})
no_recod = boletas04[boletas04['ArtCod'].str.len() != 9]
no_recod

,ArtCod,articulo
14,1113052,Roscas
15,1113053,Campechanas y Lenguas
17,1113055,Donas
18,1113061,"Milhojas, Pastelitos"
19,1113062,"Pasteles industrializados (Bimbo, Marinela, Ne..."
...,...,...
55033,1230011,Hierbas para infusiones
55070,1171051,Apio
55073,1171071,Pacaya
55075,1171091,Chipilín


In [247]:
prod_norecod = no_recod.groupby('ArtCod').value_counts().reset_index(name='Counts')
prod_norecod.sort_values('Counts', ascending=False, inplace=True)

#prod_norecod[prod_norecod['Counts'] == 147]

In [248]:
#se guardan los nombres de los productos que no se pueden recodificar a la base anterior
norecodname01 = 'sinrecodificar'
norecodname02 = '.xlsx'
norecodname = f"{norecodname01}_{mes}_{anio}{norecodname02}"

prod_norecod.to_excel(norecodname)

norecodname

'sinrecodificar_2_2024.xlsx'

In [249]:
#aqui voy a guardar los productos con sus precios para "imputarlos"
imputname01 = 'paraimputar'
imputname02 = '.xlsx'
inputname = f"{imputname01}_{mes}_{anio}{imputname02}"

artimput.to_excel(inputname, index=False)

inputname

'paraimputar_2_2024.xlsx'

In [250]:
#esta es la parte de los outliers
ignorar = False
if ignorar:
    ig = pd.read_excel(f'IGNORAR{anio}-{mes}.xlsx', dtype={f'ArtCod':str,f'RegCod':str})
    ig = ig.set_index(['RegCod','ArtCod','BolNum'])
else:
    ig = []

In [251]:
#Funcion para calcular las variaciones de cada cotización
def Variaciones(boleta, decada, outliers, ignorar = False):
    ''' 
    Esta función calcula las variaciones de la decada ingresada dado un arreglo de boletas y una decada
    
    Argumentos:
    boletas (DataFrame) : Un dataframe de las cotizaciones con las columnas RegCod, ArtCod, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem y el multiíndice (RegCod, ArtCod)
    dec (int) : La década en la que se va a calcular el índice, 

    '''
    boletas = boleta.copy()
    if decada == 0:
        semana = [1,2,3]
    elif decada == 12:
        semana = [1,2]
    elif decada == 23:
        semana =[2,3]
    else:
        semana = [decada]

    boletas = boletas.loc[boletas['PerSem'].isin(semana)]
    
    Variacion = pd.DataFrame((boletas['ArtPac']*boletas['UraChi'])/(boletas['ArtPhi']*boletas['UreCan']), columns = ['Var'])
    Variacion = Variacion.replace([np.nan,0,np.inf],1)
    Variacion.loc[boletas['ArtCR'] == 'S'] = 1.0
    if ignorar:
            Variacion.loc[Variacion.index.isin(outliers.index), 'Var'] = 1.0
    return Variacion

In [252]:
variaciones = Variaciones(boletas, dec, ig, ignorar)
variaciones

Var
RegCod ArtCod    BolNum           
01     011110101 1035-2-2024   1.0
                 1104-2-2024   1.0
                 113-2-2024    1.0
                 1175-2-2024   1.0
                 1220-2-2024   1.0
...                            ...
08     125110401 16025-2-2024  1.0
                 16027-2-2024  1.0
                 16052-2-2024  1.0
                 16435-2-2024  1.0
                 16444-2-2024  1.0

[51961 rows x 1 columns]

In [253]:
def Periodos(connect):
        '''
        Llama las periodicidades de todos los artículos
        Args:
        connect (Connection) : conexión a la base de datos
        '''
        subquery = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 \
                WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
        query = text(f"SELECT ArtCod, \
                CASE \
                WHEN ArtPRC = 5 THEN  6\
                WHEN ArtPRC = 6 THEN  12\
                WHEN ArtPRC = 1 THEN  1\
                WHEN ArtPRC = 2 THEN  2\
                WHEN ArtPRC = 3 THEN  3\
                WHEN ArtPRC = 4 THEN  4\
                ELSE 1\
                END AS ArtPRC\
                FROM IPC2010_RN.dbo.IPC007 \
                WHERE ArtCod IN \
                ({subquery})\
                ORDER BY ArtCod")
        indice = 'ArtCod'
        return pd.read_sql(query, connect, index_col=indice)

In [254]:
#Función para calcular las variaciones acumuladas de los 441 artículos
def VarAcum(var,per, cod): #esta es la funcion que ellos llaman VarFlex
    ''' 
    Esta función calcula los índices relativos (variación acumulada) de los 441 artículos para todas las regiones y retorna un arreglo con índices (region, artículo). A los artículos que no fueron cotizados les asigna variación 1

    Args
    var (DataFrame) : un dataframe con las variaciones de las fuentes con multiíndice RegCod, ArtCod
    per (DataFrame) : dataframe con las periodicidades de los artículos con índice ArtCod
    cod (DataFrame) : un dataframe con los 441 códigos de los artículos
    '''

    agrupacion = var.groupby(level = ['RegCod','ArtCod']).agg(sp.gmean)
    periodicidad = agrupacion.join(per.astype(float), how = 'outer')
    raiz = pd.DataFrame(periodicidad['Var']**(1/(periodicidad['ArtPRC'])), columns = ['Var']).sort_index()
    arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
    Codigos = cod.copy()
    Codigos['ArtCod'] = Codigos.index
    codigos2 = Codigos.merge(arreglo, how = 'cross')
    codigos2 = codigos2.set_index(['RegCod','ArtCod'])
    codigos2['Var'] = raiz['Var']
    codigos2 = codigos2.replace(np.nan, 1)
    return codigos2

In [255]:
periodos = Periodos(connection)
Varac_prod = VarAcum(variaciones, periodos, codigos)
Varac_prod

,,Var
RegCod,ArtCod,
01,011110101,1.005904
02,011110101,0.996082
03,011110101,0.991775
04,011110101,1.000099
05,011110101,1.000906
...,...,...
04,125110501,1.000000
05,125110501,1.000000
06,125110501,1.000000


In [256]:
#Se agregan las variaciones del armado nacional
def FormacionNacional(indart):
    lista = ['071110101','071110102','071110103','072420201','073210101','083110101','083110202','083110301','083110302',\
                    '083110401','093210201','093310101','095110201','123110101','123110102','123110103','123110104','124110101','125110101','125110501']
    indices = indart.copy()
    for index in lista:
        try:
            indices.loc[(indart.index.get_level_values('ArtCod') == index),'Var'] = indart.loc['01', index]['Var']
        except KeyError:
            pass
    return indices

In [257]:
Varac_prod_01 = FormacionNacional(Varac_prod).sort_index() #variacion acumulada
Varac_prod_01

Var
RegCod ArtCod             
01     011110101  1.005904
       011110102  1.000000
       011110103  1.002890
       011120101  1.013519
       011120102  0.978665
...                    ...
08     125110101  1.000000
       125110201  1.000000
       125110301  1.000000
       125110401  1.000000
       125110501  1.000000

[3528 rows x 1 columns]

In [258]:
#Función para llamar la ponderación de cualquier categoría
def CatPon(cod : str, dic, connect):
    if cod == 'Art':
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCPH6 \
                    WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    else:
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCM0{dic[cod]}")
    
    query = text(f'SELECT RegCod, {cod}Cod, {cod}Pon\
        FROM  IPC2010_RN.dbo.IPCP0{dic[cod]}\
        WHERE {cod}Cod IN ({subquery}) \
        ORDER BY RegCod, {cod}Cod')              
    return pd.read_sql(query, connect, index_col=['RegCod', cod+'Cod'])

In [259]:
#Función de cálculo de índices (relativos o absolutos)
def CatInd(cod :str, Varac_prod_01, pond, len = leng, codigo = codigos):
    pond = pond.copy()
    pond.columns = ['Pon']
    index = Varac_prod_01.copy()
    index.columns = ['Ind']
    pond[cod+'Cod'] = pond.index.get_level_values(1).map(lambda x: x[:len[cod]])
    pond = pond.set_index([pond.index.get_level_values(0), pond[cod+'Cod'], pond.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    pond[cod+'Pon'] = pond.groupby(level = ['RegCod', cod+'Cod']).sum()
    pond[cod+'Pon']=pond[cod+'Pon'].replace(0,1)
    cont = pd.DataFrame(index['Ind']*pond['Pon']/pond[cod+'Pon'], columns = [cod+'Ind'])
    cont[cod+'Cod'] = cont.index.get_level_values(1).map(lambda x: x[:len[cod]])
    cont = cont.set_index([cont.index.get_level_values(0), cont[cod+'Cod'], cont.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    catind = cont.groupby(level = ['RegCod', cod+'Cod']).sum()

    if cod == 'Gba':
        arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
        cods = CodigosCat(cod, codigo)
        Codigos = cods.copy()
        Codigos[cod+'Cod'] = Codigos.index
        codigos2 = Codigos.merge(arreglo, how = 'cross')
        codigos2 = codigos2.set_index(['RegCod',cod+'Cod'])
        codigos2[cod+'Ind'] = catind[cod+'Ind']
        codigos2 = codigos2.replace(np.nan, 1)
        return codigos2
    else:
        return catind

In [260]:
CatPon('Gba', di, connection)

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [261]:
pond = CatPon('Art', di, connection) #ponderaciones 
pond

ArtPon
RegCod ArtCod              
00     011110101   64.74764
       011110102   20.38296
       011110103   14.86940
       011120101   90.25644
       011120102    9.74356
...                     ...
08     125110101  100.00000
       125110201  100.00000
       125110301  100.00000
       125110401  100.00000
       125110501    0.00000

[3969 rows x 1 columns]

In [262]:
#Calculamos los índices relativos (variación) de gasto básico de las 8 regiones
Varprod02 = CatInd('Gba', Varac_prod_01, pond, leng)
Varprod02

,,GbaInd
RegCod,GbaCod,
01,0111101,1.003797
02,0111101,0.996368
03,0111101,0.993374
04,0111101,0.998413
05,0111101,1.000764
...,...,...
04,1251105,1.000000
05,1251105,0.000000
06,1251105,1.000000


In [263]:
#extraemos los indices anteriores
indant01 = pd.read_excel(excel_indant)
indant01.drop(indant01.columns[0], axis=1, inplace=True)
indant01['RegCod'] = indant01['RegCod'].astype(str)
indant01['RegCod'] = indant01['RegCod'].replace({'0':'00','1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07','8':'08',})
indant01['GbaCod'] = indant01['GbaCod'].astype(str)
indant01['GbaCod'] = indant01['GbaCod'].apply(lambda x: '0' + x if len(x) == 6 else x)
indant01 = indant01[indant01['RegCod'] != '00']
indant01 = indant01.set_index(['RegCod', 'GbaCod'])
indant = indant01
indant

GbaInd
RegCod GbaCod             
01     0111101  178.303114
       0111201  243.911398
       0111202  232.956673
       0111203  151.302830
       0111301  173.478221
...                    ...
08     1251101  119.542285
       1251102  105.220862
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

In [264]:
#Función para extraer índices anteriores de cualquier categoría
def IndAnt(anio : int, mes : int, cod : str, conectar, dic = di, ano = False):
    if ano:
        mesant = mes
        anioant = anio - 1
    else:
        if mes == 1:
            anioant = anio -1
            mesant = 12
        else:
            anioant = anio
            mesant = mes-1

    query = text(f'SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_01_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_02_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_03_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_04_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_05_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_06_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_07_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_08_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        ORDER BY RegCod, {cod}Cod')
    return pd.read_sql(query, conectar, index_col=['RegCod',cod+'Cod'])

In [265]:
indanioant = IndAnt(anio, mes, 'Gba', connection, ano = True)
indanioant

GbaInd
RegCod GbaCod             
01     0111101  169.222418
       0111201  231.941835
       0111202  235.052526
       0111203  141.670633
       0111301  177.217124
...                    ...
08     1241101   96.279220
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2185 rows x 1 columns]

In [266]:
indprod = pd.DataFrame(indant['GbaInd']*Varprod02['GbaInd'], columns= ['GbaInd']).replace(np.nan, 0)
indprod #este seria el indice de productos actuales

GbaInd
RegCod GbaCod             
01     0111101  178.980183
       0111201  245.836254
       0111202  240.342951
       0111203  150.858449
       0111301  172.936582
...                    ...
08     1251101  119.542285
       1251102  105.220862
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

In [267]:
#Función para llamar las ponderaciones de cada región
def RegPon(connect):
    query = text(f'SELECT RegCod, RegPon \
        FROM IPC2010_RN.dbo.IPC001 WHERE RegCod != 0')            
    return pd.read_sql(query, connect, index_col= 'RegCod')

In [268]:
#Función para calcular el índice por república
def IndRep(cod :str, index, regpond, indponde):
    indices = index.copy()
    regpon = regpond.copy()
    indpon = indponde.copy()
    indreg = indices.loc[(indices.index.get_level_values(0) != '00')]
    indpond = indpon.loc[(indpon.index.get_level_values(0) != '00')]
    reppon = pd.DataFrame(indpond[cod+'Pon']*regpon['RegPon'],columns = ['Pon'])
    sumas = reppon.groupby(level =cod+ 'Cod').sum()
    regcont = pd.DataFrame(indreg[cod+'Ind']*reppon['Pon']/sumas['Pon'],columns = [cod+'Ind'])
    indrep = regcont.groupby(level = [cod+'Cod']).sum()
    indrep['RegCod'] = '00'
    indrep = indrep.set_index([indrep['RegCod'], indrep.index]).drop(columns = ['RegCod'])
    indices.loc[indices.index.get_level_values(0) == '00'] = indrep
    return pd.concat([indrep, indreg])

In [269]:
#ponderaciones de la region a nivel republica
regpon = RegPon(connection)
regpon

,RegPon
RegCod,
01,37.34790
02,5.40404
03,7.43584
04,6.53469
05,10.96698
06,18.53248
07,9.66149
08,4.11658


In [270]:
#ponderaciones de los productos a nivel republica
gbapon = CatPon('Gba', di, connection)
gbapon

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [271]:
#indice de productos del mes anterior a nivel nacional
indprod_repant = IndRep('Gba', indant, regpon, gbapon)
indprod_repant

GbaInd
RegCod GbaCod             
00     0111101  162.174137
       0111201  225.867718
       0111202  349.099137
       0111203  142.641025
       0111301  190.264272
...                    ...
08     1251101  119.542285
       1251102  105.220862
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2511 rows x 1 columns]

In [272]:
#indice prodcutos a nivel republica del año anterior
indprod_anioant = IndRep('Gba', indanioant, regpon, gbapon)
indprod_anioant

GbaInd
RegCod GbaCod             
00     0111101  155.734609
       0111201  211.923462
       0111202  344.527703
       0111203  137.698729
       0111301  191.024568
...                    ...
08     1241101   96.279220
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2464 rows x 1 columns]

In [273]:
#indice productos a nivel republica actual
indprod_rep = IndRep('Gba', indprod, regpon, gbapon)
indprod_rep

GbaInd
RegCod GbaCod             
00     0111101  162.378833
       0111201  225.470490
       0111202  334.970322
       0111203  141.830643
       0111301  189.780546
...                    ...
08     1251101  119.542285
       1251102  105.220862
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2511 rows x 1 columns]

In [274]:
indprod_rep01 = indprod_rep.reset_index()
indprod_rep01

,RegCod,GbaCod,GbaInd
0,00,0111101,162.378833
1,00,0111201,225.470490
2,00,0111202,334.970322
3,00,0111203,141.830643
4,00,0111301,189.780546
...,...,...,...
2506,08,1251101,119.542285
2507,08,1251102,105.220862
2508,08,1251103,114.954121
2509,08,1251104,117.797094


In [275]:
indprod_rep01.to_excel(excel_indactual)

In [276]:
''' 
Aquí se pueden calcular los índices de cualquier categoría, sólo se debe cambiar el argumento Div (división) por el argumento Agr, Gru, Sub, Gba dependiendo del índice que se desee
'''
cat = 'Div'
inddiv = CatInd(cat, indprod_rep, gbapon)
pondiv = CatPon(cat, di, connection)
inddiv_rep = IndRep(cat, inddiv, regpon, pondiv)
inddiv_rep #indice de division a nivel republica actual

DivInd
RegCod DivCod            
00     01      283.554806
       02      130.057732
       03      120.320845
       04      128.867091
       05      131.523246
...                   ...
08     08      113.919847
       09      114.234165
       10      106.721121
       11      144.135792
       12      124.115640

[108 rows x 1 columns]

In [277]:
def IPC(index, pond):
    indices = index.copy()
    indices.columns = ['Ind']
    pon = pond.copy()
    pon.columns = ['Pon']
    return pd.DataFrame(indices['Ind']*pon['Pon']/100, columns = ['IPC']).groupby(level = ['RegCod']).sum()

In [278]:
'''esto seria el IPC actual'''
#IPC nacional actual
ipc_actual = IPC(inddiv, pondiv)
ipc_actual

,IPC
RegCod,
00,175.116523
01,143.522596
02,270.993444
03,161.796349
04,260.738920
05,158.568994
06,160.484443
07,241.081391
08,179.175571


In [279]:
#IPC nacional mes pasado
ipc_anterior = IPC(indprod_repant, gbapon)
ipc_anterior

,IPC
RegCod,
00,175.359426
01,143.733555
02,269.008926
03,162.393929
04,259.604856
05,159.196765
06,161.506426
07,241.136355
08,180.085600


In [280]:
ipc_anioant = IPC(indprod_anioant, gbapon)
ipc_anioant

,IPC
RegCod,
00,169.045468
01,140.938155
02,257.217276
03,158.734888
04,236.032510
05,154.913411
06,154.901719
07,232.920945
08,172.002448


Variaciones

In [281]:
#variacion intermensual
(ipc_actual-ipc_anterior)*100/ipc_anterior

,IPC
RegCod,
00,-0.138517
01,-0.146771
02,0.737715
03,-0.367982
04,0.436843
05,-0.394336
06,-0.632782
07,-0.022794
08,-0.505331


In [282]:
(ipc_actual-ipc_anioant)*100/ipc_anioant

,IPC
RegCod,
00,3.591374
01,1.833741
02,5.355849
03,1.928663
04,10.467376
05,2.359759
06,3.604043
07,3.503526
08,4.170361


Incidencias por division

In [283]:
def Incidencias(index, indexant, pond,ipcante):
    pon = pond.copy()
    pon.columns = ['Pon']
    indicesant = indexant.copy()
    indicesant.columns = ['Ind']
    indices = index.copy()
    indices.columns = ['Ind']
    return pd.DataFrame(pon['Pon']*(indices['Ind']-indicesant['Ind'])/ipcante['IPC'],columns= ['Incid'])

In [284]:
inddiv_repant = CatInd('Div', indprod_repant, gbapon)

In [285]:
incid_divrep = Incidencias(inddiv_rep, inddiv_repant, pondiv, ipc_anterior)
incid_divrep.loc[incid_divrep.index.get_level_values('RegCod') == '00']

Incid
RegCod DivCod          
00     01     -0.075897
       02      0.000223
       03      0.003294
       04     -0.018909
       05     -0.000950
       06      0.000387
       07     -0.070796
       08      0.000088
       09      0.003564
       10      0.000584
       11      0.020911
       12     -0.000958

In [286]:
incid_divrep.sort_values('Incid')

,,Incid
RegCod,DivCod,
06,01,-0.662216
08,01,-0.507562
05,01,-0.391682
01,07,-0.178396
03,01,-0.167597
...,...,...
04,11,0.051769
06,11,0.055864
05,11,0.103052


Incidencia productos

In [287]:
incid_prodrep = Incidencias(indprod_rep, indprod_repant, gbapon, ipc_anterior)
incid_prodrep.loc[incid_prodrep.index.get_level_values('RegCod') == '00']

Incid
RegCod GbaCod           
00     0111101  0.000738
       0111201 -0.000388
       0111202 -0.107700
       0111203 -0.004333
       0111301 -0.007201
...                  ...
       1251101  0.000000
       1251102  0.000000
       1251103  0.000000
       1251104  0.000171
       1251105  0.000000

[279 rows x 1 columns]

In [288]:
incid_prodrep.loc[incid_prodrep['Incid'].notnull()]

Incid
RegCod GbaCod           
00     0111101  0.000738
       0111201 -0.000388
       0111202 -0.107700
       0111203 -0.004333
       0111301 -0.007201
...                  ...
08     1251101  0.000000
       1251102  0.000000
       1251103  0.000000
       1251104  0.000000
       1251105  0.000000

[2511 rows x 1 columns]

In [289]:
ipc_actualreg = ipc_actual.loc[ipc_actual.index != '00']
ipc_antreg = ipc_anterior.loc[ipc_anterior.index != '00']
ipc_anttotal = ipc_anterior.loc['00','IPC']

In [290]:
#incid_reg = 
pd.DataFrame(regpon['RegPon']*(ipc_actualreg['IPC']-ipc_antreg['IPC'])/ipc_anttotal, columns = ['Incid'])
#incid_reg

,Incid
RegCod,
01,-0.044930
02,0.061157
03,-0.025339
04,0.042260
05,-0.039261
06,-0.108006
07,-0.003028
08,-0.021363
